In [40]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from scipy.spatial.transform import Rotation
import glob
import os
import json

In [46]:
p_space = re.compile('( *)(.*)')

def remove_space(x):
    name = p_space.match(x)
    return name.group(2)

# extracting transformation parameters (rotation angles and translation vector) from csv
def extract_transformation(x):
    # converting to rotations
    r = Rotation.from_euler('XYZ', [x['pose_Rx'], x['pose_Ry'], x['pose_Rz']], degrees=False)
    # converting to translation vector
    translation = np.array([x['pose_Tx'], x['pose_Ty'], x['pose_Tz']])
    return r, translation


# mapping global coordinate to local ones using the inverse transformation defined by the Pose parameters
def map_row_to_local_quat(x):
    r, translation = extract_transformation(x)
    
    # Each row is a (possibly non-unit norm) quaternion in scalar-last (x, y, z, w) format.
    quat = r.as_quat()
    x['quat_x'] = quat[0]  # x
    x['quat_y'] = quat[1]  # y
    x['quat_z'] = quat[2]  # z
    x['quat_w'] = quat[3]  # w
    
    # mapping each of the 68 landmarks points to local coordinates
    for i in range(0, 68):
        # converting to location vector
        vec_global = np.array([x['X_' + str(i)], x['Y_' + str(i)], x['Z_' + str(i)]])
        # translating to object space and inverting rotation
        vec_local = r.apply((vec_global - translation), inverse=True)
        # updating row with local coordinates
        x['X_L_' + str(i)] = vec_local[0]
        x['Y_L_' + str(i)] = vec_local[1]
        x['Z_L_' + str(i)] = vec_local[2]

    return x

In [47]:
# please add an absolute path to the processed directory of OpenFace
# video_path = ".\\samples\\train\\openface_landmarks - Copy.csv"
input_paths = [".\\samples\\test\\openface_landmarks.csv"]
#glob.glob(video_path + "*.csv")

# please specify a location where to save the processed CSVs
save_location = ".\\samples\\test\\jupyter_landmarks.csv" 

In [48]:
# preprocessing
for path in tqdm(paths):
    df = pd.read_csv(path)
    df = df.rename(columns=remove_space)
    df = df.apply(map_row_to_local_quat, axis=1)
    df = df.drop(['face_id', 'success', 'timestamp', 'frame','confidence',
                  'pose_Tz', 'pose_Ty', 'pose_Tx', 'pose_Rx', 'pose_Ry',
                  'pose_Rz'], axis=1)
    for i in range(0, 68):
        df = df.drop(["X_" + str(i), "Y_" + str(i), "Z_" + str(i)], axis=1)
    
    df.to_csv(save_location)

    # base_name = os.path.basename(path)
    # df.to_csv(str(save_location) + "\\" + str(base_name) + "_proc" + ".csv")
    # df.to_json(str(save_location) + "\\" + str(base_name) + "_proc" + ".json", orient="index")

In [49]:
# an example of how the CSV looks
df.head()

,X_L_0,Y_L_0,Z_L_0,X_L_1,Y_L_1,Z_L_1,X_L_2,Y_L_2,Z_L_2,X_L_3,...,X_L_66,Y_L_66,Z_L_66,X_L_67,Y_L_67,Z_L_67,quat_x,quat_y,quat_z,quat_w
0,-76.475816,-44.701913,37.948630,-77.621994,-20.443849,40.089338,-76.307887,4.975956,43.009374,-72.857893,...,-0.300993,33.853580,-10.789556,-8.907543,33.275146,-9.356832,0.077389,0.012869,0.034582,0.996318
1,-75.679900,-44.566947,37.096091,-76.942369,-20.563169,39.185302,-75.636948,4.594118,42.262113,-72.114060,...,0.105748,33.810996,-10.108185,-8.801270,33.315750,-8.565537,0.070891,0.010816,0.038345,0.996688
2,-75.872284,-44.471290,37.796175,-76.915117,-20.726985,39.696307,-75.610635,3.980467,42.627194,-72.259010,...,0.054730,34.085985,-9.826121,-9.171222,33.526593,-8.346630,0.069039,0.014313,0.039557,0.996727
3,-76.829815,-44.742632,40.215874,-77.757097,-21.069322,41.845162,-76.104907,3.773785,44.001609,-72.200580,...,0.035624,33.281583,-10.840431,-7.886997,32.804548,-9.504573,0.066187,0.017888,0.040239,0.996835
4,-78.329094,-45.625384,41.820897,-79.020896,-21.241776,43.449109,-77.049591,4.111039,45.623676,-72.517375,...,-0.009666,33.748191,-10.164128,-8.852604,33.058084,-8.702557,0.068633,0.016824,0.039718,0.996709


In [50]:
# an exmaple of how the json looks like
with open(str(save_location) + "\\" + str(base_name) + "_proc" + ".json", encoding='utf-8') as data_file:
    data = json.loads(data_file.read())
data

NameError: name 'base_name' is not defined

In [ ]:
for i in range(0, 68):
    print (["X_" + str(i), "Y_" + str(i), "Z_" + str(i)])